# Pinecone

In [1]:
#Importing libraries and environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
#Importing HuggingFace token
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [3]:
#Defining an Embedding Model
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


c:\Users\emelvis\10.VS_Code\04_AgenticAI\agenticenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Checking the working of embeddings
visakh=embeddings.embed_query("Hi This is Visakh")
len(visakh)

384

In [5]:
#Configuring Google Gemini API Key
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [6]:
#Checking the dimension of the google embdding model
len(embeddings.embed_query("Hello World"))

768

In [7]:
#importing pinecone 
from pinecone import Pinecone


In [8]:
#Loading the API key from .env 
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [9]:
#Importing Pinecone API Key 
import os 
pinecone_api_key=os.getenv("PINECONE_API_KEY")


In [10]:
#Creating pinecone instance 
pc = Pinecone(api_key=pinecone_api_key)

In [11]:
#Import Serverless from Pinecone
from pinecone import ServerlessSpec

In [12]:
#Index Name
index_name="agenticai2"


In [13]:
pc.has_index(index_name)

True

In [14]:
#Creating pinecone index 
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name, 
        dimension=768, 
        metric="cosine", 
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [15]:
#load the index 
index=pc.Index(index_name)

In [16]:
#Loading data using Langchain
from langchain_pinecone import PineconeVectorStore
vector_store=PineconeVectorStore(index=index, embedding=embeddings)

In [17]:
#Creating documents to be loaded onto Pinecone 
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},#additional info
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

In [18]:
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [19]:
#Universal identification number
uuids = [str(uuid4()) for _ in range(len(documents))]

In [20]:
uuids

['b85feeb5-112b-4163-b453-444bf5c5d8c7',
 'c947753e-9435-4aa6-a0a9-f4398a3f7eb6',
 '26a9926e-e99a-4340-b026-6d51dfceb843',
 'aa8b1cf8-3d8e-4056-874d-480db7a67a97',
 '69845d7e-7d89-4ecf-b981-ca332d404b80',
 '60cd2af4-29ae-4736-9175-01f0c8614299',
 '21cdca96-48f5-42b6-88bc-069b28aee147',
 'a2cdfb5c-23d0-41fc-b6bd-1d660e09124b',
 '194dc02b-8802-4923-a2ba-4dc900983391',
 '5d996854-5775-40a0-802c-60389b1dbc8a']

In [21]:
#Add documents using vector store 
vector_store.add_documents(documents=documents, ids=uuids)

['b85feeb5-112b-4163-b453-444bf5c5d8c7',
 'c947753e-9435-4aa6-a0a9-f4398a3f7eb6',
 '26a9926e-e99a-4340-b026-6d51dfceb843',
 'aa8b1cf8-3d8e-4056-874d-480db7a67a97',
 '69845d7e-7d89-4ecf-b981-ca332d404b80',
 '60cd2af4-29ae-4736-9175-01f0c8614299',
 '21cdca96-48f5-42b6-88bc-069b28aee147',
 'a2cdfb5c-23d0-41fc-b6bd-1d660e09124b',
 '194dc02b-8802-4923-a2ba-4dc900983391',
 '5d996854-5775-40a0-802c-60389b1dbc8a']

In [22]:
#Asking question to vector store

results=vector_store.similarity_search("What is langchain?", k=1)
results

[Document(id='81d91226-ddb1-4a9f-9c1b-cd5926aab248', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!')]

In [23]:
#Asking question to vector store

results=vector_store.similarity_search("What is langchain?", 
                                       filter={"source":"tweet"})
results

[Document(id='81d91226-ddb1-4a9f-9c1b-cd5926aab248', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='fa7a9257-5907-41f6-b9ef-50dd17416e93', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='892f1847-0678-4316-b7d0-7154fd6c7559', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='d7097758-37fc-4701-912b-abcb3d5ac1d3', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again.")]

In [24]:
#Creating a retriver 
retriver=vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7}
)

In [25]:
retriver.invoke("Langchain")

[Document(id='81d91226-ddb1-4a9f-9c1b-cd5926aab248', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='fa7a9257-5907-41f6-b9ef-50dd17416e93', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='892f1847-0678-4316-b7d0-7154fd6c7559', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='d7097758-37fc-4701-912b-abcb3d5ac1d3', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again.")]

In [26]:
#Building the model 
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [27]:
#Building Prompt 
from langchain import hub 
prompt = hub.pull("rlm/rag-prompt")

#display the prompt 
import pprint
pprint.pprint(prompt.messages)


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [28]:
#String Output Parser 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [29]:
#Create function to combine the contextual data retrived from vector 
def format_docs(docs):
    return "\n \n".join(doc.page_content for doc in docs)

In [30]:
rag_chain=(
    {"context":retriver | format_docs, "question":RunnablePassthrough()}
    |   prompt
    |   model 
    |   StrOutputParser()
)

In [31]:
rag_chain.invoke("what is llama model?")

"I don't know.  The provided text does not contain information about Llama models."

In [32]:
#Building a prompt template as part of RAG 
from langchain_core.prompts import PromptTemplate

#Building prompt 
prompt=PromptTemplate(
    template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""",
    input_variables=['context', 'question']
)

#Display prompt 
prompt 

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:")

In [33]:
prompt.invoke({"question":"what is a langchain?",
               "context":"langchain is very super framework for LLM."})

StringPromptValue(text="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: what is a langchain? \nContext: langchain is very super framework for LLM. \nAnswer:")

# Creating RAG Using PDF Reader

In [ ]:
#Importing libraries and environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
#Configuring Google Gemini Embedding model  
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [34]:
#Reading input pdf 
from langchain_community.document_loaders import PyPDFLoader

#File Path 
FILE_PATH="llama2.pdf"

#Defining the loader 
loader=PyPDFLoader(FILE_PATH)

#Performing the load 
pages=loader.load()


In [36]:
#thenumber of documents
len(pages)

77

In [37]:
#Perform Chucnking Operation
from langchain_text_splitters import RecursiveCharacterTextSplitter

#Create Chunks 
splitter=RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50
)

#Splitting the document 
split_docs = splitter.split_documents(pages)

#Checking the count or resultant documents
len(split_docs)

615

In [38]:
#Loading the data onto VectorDB (PineCone)

#Importing libraries
from pinecone import Pinecone

#Loading API Keys onto local variable
pinecone_api_key=os.getenv("PINECONE_API_KEY")

#Creating Pinecone instance 
pc = Pinecone(api_key=pinecone_api_key)

#Importing Serverless
from pinecone import ServerlessSpec

#Create Pinecone index 
index_name="agenticaipinecone"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768, 
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


In [42]:
#Capturing address infor of Pinecone to be used for Vector Loading 
index=pc.Index(index_name)

In [43]:
#Create Pinecone VectorStore
from langchain_pinecone import PineconeVectorStore
vector_store=PineconeVectorStore(
    index=index, 
    embedding=embeddings )

In [44]:
#Adding documents onto vectordb
vector_store.add_documents(documents=split_docs)

['6aed5ae5-2983-489d-b6a2-32a303e99914',
 'd30d5eb7-71bd-463e-87a4-e7b3ded1b88d',
 '426f6b4f-c378-4136-8857-953d92082466',
 '2c908a16-317c-4245-89fc-7f7e65f53264',
 'f3d1fa2f-596c-44bf-9be0-d715a62efc77',
 'a276fc35-15a4-490b-bd0a-1dea51df419d',
 '3921c173-1c0b-4ecc-baa0-5959246a2562',
 '4c706d4f-4cda-4d98-a8a8-d9d57cba1562',
 'edaf5d89-3d8d-4679-9383-c39f90bdc8d8',
 '3a3d76e4-f8a6-4471-9475-8bb3973e3fe1',
 '2d5aa95f-77ca-4321-92c5-764a3c1da23e',
 'd276ec16-802d-4c66-a343-da5f4def34c5',
 '7c052e26-dbbb-419f-b9b0-563d58734f93',
 '98e45054-4d03-41d5-a4ed-fca6aeeef1a0',
 '2b4dcd4f-75da-4392-9c02-558ad1c2574d',
 'b173b5ed-6ca6-405e-8845-dd3e989fd1bf',
 '4dead6ff-5a2a-4f2f-9ee2-2e3171f65788',
 '9d434519-5152-40cb-ad48-a141b2ada02b',
 '650ba337-324b-4e6a-bed5-6572f97a5676',
 '90c268ff-7d06-4532-80bf-29a036cb5c34',
 'ead8f006-7170-4d2d-8c90-6b24327e8b61',
 'a046fe39-e63c-48b4-b48d-75154ac3ab7c',
 '0f697ac6-2543-4e7c-bf9b-60ed938097c2',
 '6b4a14ee-1de4-439e-968c-20f69d3f596a',
 '2d73d2a3-ea49-

In [45]:
#Understand the statistics of loaded data 
stats = index.describe_index_stats()
print(stats)

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 615}},
 'total_vector_count': 615,
 'vector_type': 'dense'}


In [47]:
#Create a retriver object for RAG
retriver=vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7}
)

In [48]:
#Building the model 
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [49]:
#BUilding a prompt 
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

#display the prompt 
import pprint 
pprint.pprint(prompt)

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])


In [50]:
#Creating output pharser 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#Function to combine all contextual documents
def format_docs(docs):
    return " \n \n".join(doc.page_content for doc in docs)

In [55]:
#Creating a Rag Chain
rag_chain = (
    {"context":retriver | format_docs, "question":RunnablePassthrough()}
    |   prompt 
    |   model 
    |   StrOutputParser()
)

In [56]:
rag_chain.invoke("what is llama model?")

'Based on the provided text, Llama is a large language model.  There are different versions, including Llama 1 and Llama 2, with varying sizes (e.g., 7B, 13B parameters).  More information can be found in the linked Responsible Use Guide.'